### 利用pydicom处理。使得CT的第一张对应PET的第一张,...,CT的最后一张对应PET的最后一张。即文件命名为001，002，003，...的递增序列。001就对应人体的头部（ImagePositionPatient[2]递减）。除了修改文件名，也要把DICOM中元信息Instancenumber修改为与文件名一致的顺序。就地修改。

In [6]:
import numpy as np
import os 
import sys 
import shutil
import pydicom

In [7]:
data_path = "F:\\data\\yuanfazhao\\yuanfazhao_data_04"

In [8]:
# 获取病例名字
name_path = os.listdir(data_path)
print(name_path)
print(len(name_path))

['N0-1978-wangyunqing', 'N0-2115-tangliru', 'N0-2201-ruanguanfu', 'N0-484-wulong', 'N0-485-wulingbo', 'N0-60-wangguiping', 'N1-2525-liwanfeng', 'N1-2621-jinyanlai', 'N2-1062-huangxueqi', 'N2-1901-xiafafu', 'N2-2403-liushaogang', 'N2-2506-lixiuqing', 'N2-541-wangwuping', 'N2-557-wanglijuan', 'N2-607-wanhongliang']
15


In [9]:
# 获得所有的CT、PET完整的目录
ct_path = []
pet_path = []
for v in name_path:
    temp = os.listdir(os.path.join(data_path,v))
    ct_path.append(os.path.join(data_path,v,temp[0]))
    pet_path.append(os.path.join(data_path,v,temp[2]))

# print(ct_path)
# print(pet_path)

0
F:\data\yuanfazhao\yuanfazhao_data_04\N0-1978-wangyunqing\C4NEH5E20\205.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-1978-wangyunqing\POJZYTLRB\205.dcm
1
F:\data\yuanfazhao\yuanfazhao_data_04\N0-2115-tangliru\CZPJUFO1Y\287.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-2115-tangliru\PVX1ENSN0\287.dcm
2
F:\data\yuanfazhao\yuanfazhao_data_04\N0-2201-ruanguanfu\CYPYNZS0T\205.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-2201-ruanguanfu\PE0PVNFFH\205.dcm
3
F:\data\yuanfazhao\yuanfazhao_data_04\N0-484-wulong\CE1VOZRHT\239.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-484-wulong\PERNUYMBW\239.dcm
4
F:\data\yuanfazhao\yuanfazhao_data_04\N0-485-wulingbo\CZXMCE2E0\287.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-485-wulingbo\PRJCRYXPL\287.dcm
5
F:\data\yuanfazhao\yuanfazhao_data_04\N0-60-wangguiping\C5YGD35LZ\287.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-60-wangguiping\PFTBQFDSZ\287.dcm
6
F:\data\yuanfazhao\yuanfazhao_data_04\N1-2525-liwanfeng\CWJREMFFT\239.dcm	F:\data\yuanfazhao\yuanfazhao_da

0
F:\data\yuanfazhao\yuanfazhao_data_04\N0-1978-wangyunqing\C4NEH5E20\001.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-1978-wangyunqing\POJZYTLRB\001.dcm
1
F:\data\yuanfazhao\yuanfazhao_data_04\N0-2115-tangliru\CZPJUFO1Y\001.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-2115-tangliru\PVX1ENSN0\001.dcm
2
F:\data\yuanfazhao\yuanfazhao_data_04\N0-2201-ruanguanfu\CYPYNZS0T\001.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-2201-ruanguanfu\PE0PVNFFH\001.dcm
3
F:\data\yuanfazhao\yuanfazhao_data_04\N0-484-wulong\CE1VOZRHT\001.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-484-wulong\PERNUYMBW\001.dcm
4
F:\data\yuanfazhao\yuanfazhao_data_04\N0-485-wulingbo\CZXMCE2E0\001.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-485-wulingbo\PRJCRYXPL\001.dcm
5
F:\data\yuanfazhao\yuanfazhao_data_04\N0-60-wangguiping\C5YGD35LZ\001.dcm	F:\data\yuanfazhao\yuanfazhao_data_04\N0-60-wangguiping\PFTBQFDSZ\001.dcm
6
F:\data\yuanfazhao\yuanfazhao_data_04\N1-2525-liwanfeng\CWJREMFFT\001.dcm	F:\data\yuanfazhao\yuanfazhao_da

In [ ]:
"""检查顺序，并返回按imagePositionPatient[2]值从小到大排列的3D图像"""
# path为病人的地址
def check_order(path):
    
    # 保存{目录：数据}
    name_slices = {}
    
    for s in os.listdir(path):
        dir = os.path.join(path,s)
        slice = pydicom.dcmread(dir)
        name_slices[dir] = slice
    
    # 排序，从小到大排序
    sorted_slices = sorted(name_slices.items(),key = lambda x : float(x[1].ImagePositionPatient[2]),reverse=True)

    return sorted_slices

# 输出CT与PET排序后第一张图片的名称
for i in range(len(name_path)):
    ct_sorted = check_order(ct_path[i])
    pet_sorted = check_order(pet_path[i])
    print(i)
    print(ct_sorted[0][0] + "\t" + pet_sorted[0][0])
    
## 目的是检查CT和PET的文件名命名顺序是否一致。果然有一部分一致，一部分不一致。下面就对CT和PET按同一个文件名命名顺序进行修改。

#### 重要代码

重新编号

In [12]:
# 按照ImagePositionPatient[2]从大到小（即是从头到脚的顺序排序）编号001-***
# path为病人的地址
def rename(path):
    name_slices = {}
    
    filenames = os.listdir(path)
    
    num = len(filenames)
    
    for f in filenames:
        dir = os.path.join(path,f)
        slice = pydicom.dcmread(dir)
        name_slices[dir] = slice
    
    # 排序，从大到小排序
    sorted_slices = sorted(name_slices.items(),key = lambda x : float(x[1].ImagePositionPatient[2]),reverse=True)
    
    
    # 重命名
    # 数字前补0创建成001，002...
    i = 1
    for j in range(num):
        k = "%03d" % i
        new_filename = str(k) + ".dcm"
        os.rename(sorted_slices[j][0],os.path.join(path,new_filename))
        i = i+1
        
    return sorted_slices

# 执行重命名
for i in range(len(name_path)):
    ct_sorted = rename(ct_path[i])
    pet_sorted = rename(pet_path[i])
    print(i)
    print(ct_sorted[0][0] + "\t" + pet_sorted[0][0])
    
# 至此重命名结束。PET和CT拥有同一个文件名命名顺序，能够一一对应上。

In [7]:
print(pet_path[0].split("\\")[3].split("-")[1])

1978


In [12]:
# 就只有部分PET的是相反的
# 人工找出来的相反序列？下次换个方式吧

# 创建两个pet正序和反序的序列
pet_path_p = [] # 此序列中文件名与instance number相同
pet_path_n = [] # 此序列中文件名与instance number相反

for p in pet_path:
    s = p.split("\\")[4].split("-")[1]
    if s in ["1978","2201","484","2525","2403","2506","557","607"]:
        pet_path_n.append(p)
    else:
        pet_path_p.append(p)

print(pet_path_n)
print("\n")
print(pet_path_p)

['F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N0-1978-wangyunqing\\POJZYTLRB', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N0-2201-ruanguanfu\\PE0PVNFFH', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N0-484-wulong\\PERNUYMBW', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N1-2525-liwanfeng\\P4ZLYZYPL', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N2-2403-liushaogang\\PLZCXHYGF', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N2-2506-lixiuqing\\PUUSM4RWV', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N2-557-wanglijuan\\PNYYC2GLE', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N2-607-wanhongliang\\P402CVNYW']


['F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N0-2115-tangliru\\PVX1ENSN0', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N0-485-wulingbo\\PRJCRYXPL', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N0-60-wangguiping\\PFTBQFDSZ', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N1-2621-jinyanlai\\PPOVGQPRV', 'F:\\data\\yuanfazhao\\yuanfazhao_data_04\\N2-1062-huangxueqi\\PNVXAJ4EX', 'F:\\data\\yuanfazhao\\yuanfazhao

 ### 此序列中文件名与instance number相反
['F:\\data\\yuanfazhao_data_04\\N0-1978-wangyunqing\\POJZYTLRB',

 'F:\\data\\yuanfazhao_data_04\\N0-2201-ruanguanfu\\PE0PVNFFH', 
 
 'F:\\data\\yuanfazhao_data_04\\N0-484-wulong\\PERNUYMBW', 
 
 'F:\\data\\yuanfazhao_data_04\\N1-2525-liwanfeng\\P4ZLYZYPL', 
 
 'F:\\data\\yuanfazhao_data_04\\N2-2403-liushaogang\\PLZCXHYGF', 
 
 'F:\\data\\yuanfazhao_data_04\\N2-2506-lixiuqing\\PUUSM4RWV', 
 
 'F:\\data\\yuanfazhao_data_04\\N2-557-wanglijuan\\PNYYC2GLE', 
 
 'F:\\data\\yuanfazhao_data_04\\N2-607-wanhongliang\\P402CVNYW'] 
 
### 此序列中文件名与instance number相同
['F:\\data\\yuanfazhao_data_04\\N0-2115-tangliru\\PVX1ENSN0',

 'F:\\data\\yuanfazhao_data_04\\N0-485-wulingbo\\PRJCRYXPL', 
 
 'F:\\data\\yuanfazhao_data_04\\N0-60-wangguiping\\PFTBQFDSZ', 
 
 'F:\\data\\yuanfazhao_data_04\\N1-2621-jinyanlai\\PPOVGQPRV', 
 
 'F:\\data\\yuanfazhao_data_04\\N2-1062-huangxueqi\\PNVXAJ4EX', 
 
 'F:\\data\\yuanfazhao_data_04\\N2-1901-xiafafu\\P5CBOVZA2', 
 
 'F:\\data\\yuanfazhao_data_04\\N2-541-wangwuping\\P3LOFW1US']

####  重要代码
修改instance number

In [17]:
# 修改Instance Number的函数
# path为病人地址
def alter_instancenumber(path):
    
    name_slices = {}
    
    filenames = os.listdir(path)
    
    num = len(filenames)
    
    for f in filenames:
        dir = os.path.join(path,f)
        slice = pydicom.dcmread(dir)
        name_slices[dir] = slice
    
    # 排序，从大到小排序
    sorted_slices = sorted(name_slices.items(),key = lambda x : float(x[1].ImagePositionPatient[2]),reverse=True)
   
    for i in range(num):
        v = str(i+1)
        # 修改并保存
        sorted_slices[i][1].InstanceNumber = v
        sorted_slices[i][1].save_as(sorted_slices[i][0])
    
    return num

l = len(pet_path_n)
print(l)

# 执行修改
for i in range(l):
    num = alter_instancenumber(pet_path_n[i])
    print(pet_path_n[i],"\t",num)
    
# 检查修改后的instance number
for i in range(l):
    pet_sorted = check_order(pet_path_n[i])
    print(i+1)
    print(pet_sorted[0][0],"\t",pet_sorted[0][1].ImagePositionPatient[2])